One last time we go through all of the setup required but stop at making a final prediction layer. This final prediction layer is a neural layer, and as such requires a long time to train. Unfortunately we don't have that many test images to make this layer worth while, so we instead use an SVM for classification!

In [1]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

images_path = os.getcwd()
print(images_path)

def refresh_dir(path, name):
    try:
        shutil.rmtree(path + name)
    except Exception as e:
        print("Couldn't remove " + str(path) + str(name))
    finally:
        os.mkdir(path + name)

refresh_directories = ["/Images-Using/Other-Usable"]
for ref_dir in refresh_directories:
    refresh_dir(images_path, ref_dir)
    
def do_splits(src, directory_given, class_idx):
    directory = images_path + directory_given
    allFileNames = os.listdir(directory)
    allFileNames = [src + directory_given + '/' + name for name in allFileNames]
    directory = "/Images-Using/Other-Usable"

    idx = 0
    for name in allFileNames:
        failed_naming = True
        while failed_naming:
            try:
                shutil.copy(name, src + directory)
                os.rename(src + directory + '/' + name.rsplit('/', 1)[-1],
                          src + directory + '/' + str(class_idx) + '-' + str(idx) + name.rsplit('/', 1)[-1])
                failed_naming = False
            except:
                idx += 1
            idx += 1

classes = ["/Human", "/Nature-Background", "/Text", "/Dogs", "/Cats", "/Hamsters", "/Suggestively-Sexual",
           "/Suggestively-Violent", "/Abstract-Background", "/Empty-Cages", "/Bunny-Drawings"]
class_idx = 0
for class_dir in classes:
    do_splits(images_path, "/Other-Directory" + class_dir, class_idx)
    class_idx += 1

batch_size = 1
img_height = 160
img_width = 160

split_directory = images_path + "/Images-Using"

train_ds = tf.keras.utils.image_dataset_from_directory(
    split_directory,
    validation_split=0.2,
    subset="training",
    seed=321,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    split_directory,
    validation_split=0.2,
    subset="validation",
    seed=321,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_batches = tf.data.experimental.cardinality(validation_ds)
test_ds = validation_ds.take(val_batches // 4)
validation_ds = validation_ds.skip(val_batches // 4)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

IMG_SIZE = (160, 160)

IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

base_model.trainable = False

/Users/Alex/Desktop/Image-Classification
Found 1222 files belonging to 2 classes.
Using 978 files for training.
Metal device set to: Apple M1 Pro
Found 1222 files belonging to 2 classes.
Using 244 files for validation.


2022-10-13 13:49:21.662514: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-13 13:49:21.662837: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-10-13 13:49:22.547602: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


How will this SVM work? And on what? We will use the activation layer of our neural network to feed into an SVM. We'll test out a few layers and see how they perform!

In [2]:
print(base_model.summary())

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 80, 80, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 80, 80, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 80, 80, 32)   0           ['bn_Conv1[0][

 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 20, 20, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_depthwise_relu (ReLU)  (None, 20, 20, 144)  0           ['block_3_depthwise_BN[0][0]']   
                                                                                                  
 block_3_project (Conv2D)       (None, 20, 20, 32)   4608        ['block_3_depthwise_relu[0][0]'] 
                                                                                                  
 block_3_project_BN (BatchNorma  (None, 20, 20, 32)  128         ['block_3_project[0][0]']        
 lization)

 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 10, 10, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 10, 10, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)                                                                                            
                                                                                                  
 block_7_depthwise_BN (BatchNor  (None, 10, 10, 384)  1536       ['block_7_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_7_d

 block_10_project_BN (BatchNorm  (None, 10, 10, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 10, 10, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_expand_BN (BatchNorma  (None, 10, 10, 576)  2304       ['block_11_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_11_expand_relu (ReLU)    (None, 10, 10, 576)  0           ['block_11_expand_BN[0][0]']     
                                                                                                  
 block_11_

                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 5, 5, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 5, 5, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_project_BN (BatchNorm  (None, 5, 5, 160)   640         ['block_14_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_14_add (Add)             (None, 5, 5, 160)    0           ['block_13_project_BN[0][0]',    
                                                                  'block_14_project_BN[0][0]']    
          

In [3]:
X = np.array([[-4,-1],[-3, 1],[-3, 3],[-2, 1],[-2, 2],[-1, 2],[-1, 3],[2, 4], [4, 4], [4, 5], [4, -2], [-2, 3],[-2, -4],[-1, -3],[2, 3],[2, -3],[2, -4],[3, 1],[3, -1],[3, -2],[3, -3],[4, 1]])
y = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(), SVC(gamma="auto"))
clf.fit(X, y)

print(clf.predict([[-1, 5]]))

[1]


In [4]:
from keras.models import Model

print(len(list(train_ds)))

x_train = np.empty((0, 5, 5, 1280), float)
y_train = np.empty((0), int)

intermediate_layer_model = Model(inputs=base_model.input,
                                outputs=base_model.get_layer('out_relu').output)
    
for image, label in train_ds:
    
    intermediate_output = intermediate_layer_model.predict(image)
    x_train = np.concatenate((x_train, intermediate_output), axis=0)
    y_train = np.concatenate((y_train, label), axis=0)
    
print(x_train)
print(y_train)

978


2022-10-13 13:49:23.689782: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 16ms/step
[[[[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         1.13638961 0.         ... 0.         0.
    0.        ]
   [0.         0.94659638 0.         ... 0.         0.1266465
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         2.04409957 0.         ... 0.         1.02097893
    0.        ]
   [0.         1.11547863 0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         1.3032707  0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.

In [5]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

n_rows, nx, ny, nz = x_train.shape
x_train = x_train.reshape((n_rows,nx*ny*nz))
x_train = np.nan_to_num(x_train, nan=0)

clf = make_pipeline(StandardScaler(), SVC(gamma="auto"))
clf.fit(x_train, y_train)

KeyboardInterrupt: 

In [ ]:
correct = 0
wrong = 0

for image, label in test_ds:
    intermediate_output = intermediate_layer_model.predict(image)
    # print(clf.predict(intermediate_output.reshape((1,nx*ny*nz))))
    # print(label)
    if(clf.predict(intermediate_output.reshape((1,nx*ny*nz))) == label):
        correct += 1
    else:
        wrong += 1

print("accuracy: " + str(correct / (correct + wrong)))

In [7]:
!conda install --yes umap-learn

Solving environment: done

## Package Plan ##

  environment location: /Users/Alex/opt/miniconda3

  added / updated specs:
    - umap-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2022.9.24          |   py39hca03da5_0         155 KB
    ------------------------------------------------------------
                                           Total:         155 KB

The following packages will be UPDATED:

  certifi                          2022.9.14-py39hca03da5_0 --> 2022.9.24-py39hca03da5_0 None



certifi-2022.9.24    | 155 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done


In [ ]:
import umap

reducer = umap.UMAP()

print(x_train.shape)
print("test")

In [63]:
print(x_train_please.shape)
print(y_train.shape)
print(x_train[-1])
print(y_train[-1])

(979, 32000)
(978,)
[[[0.         2.09364915 0.         ... 0.         0.         0.        ]
  [0.         2.41794038 0.         ... 0.         0.         0.        ]
  [0.         4.69880915 0.         ... 0.         0.         0.        ]
  [0.         2.26949596 0.         ... 0.         0.         0.1144579 ]
  [0.         0.85044163 0.         ... 0.         0.         0.73296714]]

 [[0.         0.38994443 0.         ... 0.         0.         0.48430383]
  [0.         1.32728112 0.         ... 0.         0.         1.39676523]
  [3.29929328 4.02865028 0.         ... 0.         0.         4.31988668]
  [0.26550177 2.67171335 0.         ... 0.         0.         2.89614773]
  [0.         0.84360981 0.         ... 0.         0.         1.39028585]]

 [[0.         1.24881268 0.         ... 0.         0.         1.88673055]
  [0.         2.34401965 0.         ... 0.         0.         3.54670715]
  [1.47907913 1.4941926  0.         ... 0.         0.         5.18263769]
  [0.77658677 

In [11]:
n_rows, nx, ny, nz = base_model.get_layer('out_relu').output_shape
print(n_rows)
print(nx)
print(ny)
print(nz)

None
5
5
1280


In [12]:
n_rows, nx, ny, nz = x_train.shape
x_train = x_train.reshape((n_rows,nx*ny*nz))
x_train = np.nan_to_num(x_train, nan=0)

ValueError: not enough values to unpack (expected 4, got 2)